In [1]:
import os
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Input, Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
import numpy as np

/Users/atsushi-fukuda/.local/share/virtualenvs/vgg16_fine_tuning-SDJiAvxL/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def save_history(history, result_file):
    loss = history.history['loss']
    acc = history.history['acc']
    val_loss = history.history['val_loss']
    val_acc = history.history['val_acc']
    nb_epoch = len(acc)

    with open(result_file, "w") as fp:
        fp.write("epoch\tloss\tacc\tval_loss\tval_acc\n")
        for i in range(nb_epoch):
            fp.write("%d\t%f\t%f\t%f\t%f\n" % (i, loss[i], acc[i], val_loss[i], val_acc[i]))

In [3]:
classes = ['Tulip', 'Snowdrop', 'LilyValley', 'Bluebell', 'Crocus',
           'Iris', 'Tigerlily', 'Daffodil', 'Fritillary', 'Sunflower',
           'Daisy', 'ColtsFoot', 'Dandelion', 'Cowslip', 'Buttercup',
           'Windflower', 'Pansy']

batch_size = 32
nb_classes = len(classes)

img_rows, img_cols = 150, 150
channels = 3

train_data_dir = 'train_images'
validation_data_dir = 'test_images'

# 17カテゴリあり、各カテゴリに80種類ずつの画像がある 17*80=1360
# 1360からテスト用の画像 170を引いて、1190個
nb_train_samples = 1190
# 全カテゴリにおいて10個の画像をランダムに抽出
nb_val_samples = 170
nb_epoch = 50

steps_per_epoch =  int(nb_train_samples/batch_size)

In [4]:
result_dir = 'results'
if not os.path.exists(result_dir):
    os.mkdir(result_dir)

In [5]:
# VGG16モデルと学習済み重みをロード
# Fully-connected層（FC）はいらないのでinclude_top=False）
input_tensor = Input(shape=(img_rows, img_cols, 3))
vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)

In [6]:
# 全結合層を構築
# Flattenへの入力指定はバッチ数を除く
top_model = Sequential()
top_model.add(Flatten(input_shape=vgg16.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(nb_classes, activation='softmax'))

In [7]:
# VGG16と全結合層を接続
model = Model(inputs=vgg16.input, outputs=top_model(vgg16.output))

# 最後のconv層の直前までの層は、学習済のため、学習させない
for layer in model.layers[:15]:
    layer.trainable = False

optimizer_adam = optimizers.Adam(lr=1e-4)
optimizer_sgd = optimizers.SGD(lr=1e-4, momentum=0.9)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer_adam,
              metrics=['accuracy'])

# テスト用の画像の生成
http://aidiary.hatenablog.com/entry/20161212/1481549365

In [8]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1.0 / 255)

 # サブディレクトリをクラス名にしておくと、テスト用の画像を生成してくれる
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_rows, img_cols),
    color_mode='rgb',
    classes=classes,
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_rows, img_cols),
    color_mode='rgb',
    classes=classes,
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True)

Found 1190 images belonging to 17 classes.
Found 170 images belonging to 17 classes.


# 学習の実施

In [9]:
# Fine-tuning
history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=nb_epoch,
    validation_data=validation_generator,
    validation_steps=nb_val_samples)

Epoch 1/50
37/37 [==============================] - 770s 21s/step - loss: 2.2956 - acc: 0.2708 - val_loss: 1.2620 - val_acc: 0.6414
Epoch 2/50
37/37 [==============================] - 740s 20s/step - loss: 1.1583 - acc: 0.6434 - val_loss: 0.7519 - val_acc: 0.7709
Epoch 3/50
37/37 [==============================] - 686s 19s/step - loss: 0.6684 - acc: 0.8063 - val_loss: 0.5587 - val_acc: 0.8350
Epoch 4/50
37/37 [==============================] - 688s 19s/step - loss: 0.4546 - acc: 0.8555 - val_loss: 0.5328 - val_acc: 0.8456
Epoch 5/50
37/37 [==============================] - 720s 19s/step - loss: 0.3913 - acc: 0.8825 - val_loss: 0.3829 - val_acc: 0.8939
Epoch 6/50
37/37 [==============================] - 723s 20s/step - loss: 0.2707 - acc: 0.9212 - val_loss: 0.4277 - val_acc: 0.8702
Epoch 7/50
37/37 [==============================] - 3373s 91s/step - loss: 0.2291 - acc: 0.9349 - val_loss: 0.3776 - val_acc: 0.8943
Epoch 8/50
37/37 [==============================] - 727s 20s/step - loss: 0

KeyboardInterrupt: 

In [ ]:
model.save_weights(os.path.join(result_dir, 'finetuning.h5'))
save_history(history, os.path.join(result_dir, 'history_finetuning.txt'))